In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import re
%matplotlib inline

Get data and convert it into a pandas dataframe

In [2]:
dir = r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/"
train = r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/train.json"
test = r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/test.json"


data_file = open(train, 'r')
data = json.load(data_file)
data_file.close()

df = pd.DataFrame(data)

In [3]:
print(df.columns)

Index(['bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'features', 'interest_level', 'latitude',
       'listing_id', 'longitude', 'manager_id', 'photos', 'price',
       'street_address'],
      dtype='object')


Get the features column. Remove - and double \*. Then use \* to split the features and combine all of them into a list. Convert it into a series at the end.

In [4]:
features = df['features'].tolist()
features_list = []
for feature in features:
    temp = []
    if feature != []:
        for f in feature:
            f = f.lower()
            f = re.sub("[-]", "", f)
            if ' * ' in f:    
                f = f.split(' * ')
                for element in f:
                    element = re.sub("[**]", "", element)
                    temp.append(element.strip())
            else:
                temp.append(f.strip())    
    features_list.append(temp)
features = pd.Series(features_list)

In [5]:
print(features.shape)

(49352,)


In [6]:
flattenFeatures = pd.Series([l.strip() for subL in features_list for l in subL])

In [7]:
uniqueFeatures = flattenFeatures.drop_duplicates()
len(uniqueFeatures)

1438

In [8]:
g = flattenFeatures.value_counts()
keepFeatures = g.index

In [9]:
featuresdf = pd.DataFrame(columns = keepFeatures)
featuresdf['features'] = features
featuresdf['listing_id'] = pd.Series(df['listing_id'].tolist())
print(featuresdf.shape)
print(featuresdf.columns)

(49352, 1440)
Index(['elevator', 'hardwood floors', 'cats allowed', 'dogs allowed',
       'doorman', 'dishwasher', 'laundry in building', 'no fee',
       'fitness center', 'prewar',
       ...
       'bluetooth system', 'near columbia u', '3br', 'custom painted murals',
       'featuring a sophisticated precor® online profile system that tracks your personal fitness goals',
       'high ceilings.', 'a true top luxury hirise', 'air conditioned',
       'features', 'listing_id'],
      dtype='object', length=1440)


In [10]:
#featuresdf = featuresdf.iloc[1:11,] 

** This part puts 1 for the features that exist for that listing **

In [ ]:
featuresdfc = featuresdf.copy()

for index, row in featuresdf.iterrows():
    f = list(row[-2])
    for f1 in f:
        featuresdfc.loc[index, f] = 1

In [ ]:
df_sub = df[['listing_id','bathrooms', 'bedrooms', 'building_id', 'created', 'description',
       'display_address', 'interest_level', 'latitude',
       'listing_id', 'longitude', 'manager_id', 'photos', 'price',
       'street_address']]

In [ ]:
featuresdfc.to_csv(r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/featuresdf.csv")

In [ ]:
pd.merge(df_sub, featuresdfc, on = 'listing_id')
df_sub.to_csv(r"C:/Users/sandh/Dropbox/FRM/Bootcamp/Projects/Project 3/twosigma/train.json/features_split.csv")

In [ ]:
managers_interest = df[['manager_id', 'interest_level']]
managers = df['manager_id']

In [ ]:
managers_interest_group = managers_interest.groupby(by = ['manager_id','interest_level'])

In [ ]:
(managers.drop_duplicates()).shape

In [ ]:
managers_counts = managers.value_counts()
managers_counts

In [ ]:
managers_counts.plot()